In [8]:
import re
import pandas as pd

def extract_params_from_file_name(df: pd.DataFrame):
    df['maxNewTokensFactor'] = df['file'].apply(lambda x: re.search(r'maxNewTokensFactor(\d+)', x).group(1))
    df['nShotsInference'] = df['file'].apply(lambda x: re.search(r'nShotsInference(\d+)', x).group(1))
    #df['layer'] = df['file'].apply(lambda x: re.search(r'adapters_(\s+)', x).group(1))
    df['model'] = df['file'].apply(lambda x: str(x.split('_adapters_')[0].split('nShotsInference')[1][2:]))
    df['training_params_string'] = df['file'].apply(lambda x: x.split('adapters_')[1])
    df['nbit'] = df['training_params_string'].apply(lambda x: int(x.split('_')[1]))
    df['bnb_4bit_compute_dtype'] = df['training_params_string'].apply(lambda x: x.split('_')[2])
    df['r'] = df['training_params_string'].apply(lambda x: int(x.split('_')[3]))
    df['lora_alpha'] = df['training_params_string'].apply(lambda x: int(x.split('_')[4]))
    df['lora_dropout'] = df['training_params_string'].apply(lambda x: float(x.split('_')[5]))
    df['gradient_accumulation_steps'] = df['training_params_string'].apply(lambda x: int(x.split('_')[6]))
    df['learning_rate'] = df['training_params_string'].apply(lambda x: float(x.split('_')[7].split('.')[0]))
    df = df.drop(columns=['training_params_string', 'file'])
    return df


In [8]:
llama_13B_4bit_base_wordsLevelTrue_evaluation = pd.read_csv("data/evaluation_results/llama_13B_4bit_base_wordsLevelTrue_evaluation.csv")
llama_13B_8bit_base_wordsLevelTrue_evaluation = pd.read_csv("data/evaluation_results/llama_13B_8bit_base_wordsLevelTrue_evaluation.csv")
llama_7B_4bit_base_wordsLevelTrue_evaluation = pd.read_csv("data/evaluation_results/llama_7B_4bit_base_wordsLevelTrue_evaluation.csv")
llama_7B_8bit_base_wordsLevelTrue_evaluation = pd.read_csv("data/evaluation_results/llama_7B_8bit_base_wordsLevelTrue_evaluation.csv")
llama_7B_8bit_wordsLevelTrue_evaluation = pd.read_csv("data/evaluation_results/llama_7B_8bit_wordsLevelTrue_evaluation.csv")
mistral_4bit_base_wordsLevelTrue_evaluation = pd.read_csv("data/evaluation_results/mistral_4bit_base_wordsLevelTrue_evaluation.csv")
mistral_4bit_wordsLevelTrue_evaluation = pd.read_csv("data/evaluation_results/mistral_4bit_wordsLevelTrue_evaluation.csv")
mistral_noInstr_4bit_wordsLevelTrue_evaluation = pd.read_csv("data/evaluation_results/mistral_noInstr_4bit_wordsLevelTrue_evaluation.csv")

# llama_13B_4bit_base_wordsLevelTrue_evaluation = extract_params_from_file_name(llama_13B_4bit_base_wordsLevelTrue_evaluation)
# llama_13B_8bit_base_wordsLevelTrue_evaluation = extract_params_from_file_name(llama_13B_8bit_base_wordsLevelTrue_evaluation)
# llama_7B_4bit_base_wordsLevelTrue_evaluation = extract_params_from_file_name(llama_7B_4bit_base_wordsLevelTrue_evaluation)
# llama_7B_8bit_base_wordsLevelTrue_evaluation = extract_params_from_file_name(llama_7B_8bit_base_wordsLevelTrue_evaluation)
# llama_7B_8bit_wordsLevelTrue_evaluation = extract_params_from_file_name(llama_7B_8bit_wordsLevelTrue_evaluation)
# mistral_4bit_base_wordsLevelTrue_evaluation = extract_params_from_file_name(mistral_4bit_base_wordsLevelTrue_evaluation)
# mistral_4bit_wordsLevelTrue_evaluation = extract_params_from_file_name(mistral_4bit_wordsLevelTrue_evaluation)
# mistral_noInstr_4bit_wordsLevelTrue_evaluation = extract_params_from_file_name(mistral_noInstr_4bit_wordsLevelTrue_evaluation)

res = pd.concat([llama_13B_4bit_base_wordsLevelTrue_evaluation, llama_13B_8bit_base_wordsLevelTrue_evaluation,
           llama_7B_4bit_base_wordsLevelTrue_evaluation, llama_7B_8bit_base_wordsLevelTrue_evaluation,
           llama_7B_8bit_wordsLevelTrue_evaluation, mistral_4bit_base_wordsLevelTrue_evaluation,
           mistral_4bit_wordsLevelTrue_evaluation, mistral_noInstr_4bit_wordsLevelTrue_evaluation])#.to_csv('data/evaluation_results/joint_results.csv')

In [1]:
import glob
import pandas as pd
import re

def extract_params_from_file_name(df: pd.DataFrame):
    df['model_type'] = df['file'].apply(lambda x: str(x.split('/')[1]))
    df['model_configurations'] = df['file'].apply(lambda x: str(x.split('/')[2]))
    if df['model_type'][0] == 'mistral':
        df['model_size'] = 7
        df['quantization'] = df['model_configurations'].apply(lambda x: str(x.split('_')[0]))
        try:
            df['fine_tuning'] = df['model_configurations'].apply(lambda x: str(x.split('_')[1]))
        except IndexError:
            df['fine_tuning'] = 'FT'
    else:
        df['model_size'] = df['model_configurations'].apply(lambda x: str(x.split('_')[0]))
        df['quantization'] = df['model_configurations'].apply(lambda x: str(x.split('_')[1]))
        df['fine_tuning'] = df['model_configurations'].apply(lambda x: str(x.split('_')[2]))

    df['maxNewTokensFactor'] = df['file'].apply(lambda x: re.search(r'maxNewTokensFactor(\d+)', x).group(1))
    df['nShotsInference'] = df['file'].apply(lambda x: re.search(r'nShotsInference(\d+)', x).group(1))
    #
    # df['layer'] = df['file'].apply(lambda x: re.search(r'adapters_(\s+)', x).group(1))
    df['model'] = df['file'].apply(lambda x: str(x.split('_adapters_')[0].split('nShotsInference')[1][2:].replace('.csv', '')))
    if df['fine_tuning'][0] == 'FT':
        df['training_params_string'] = df['file'].apply(lambda x: x.split('adapters_')[1])
        df['nbit'] = df['training_params_string'].apply(lambda x: x.split('_')[1])
        df['bnb_4bit_compute_dtype'] = df['training_params_string'].apply(lambda x: x.split('_')[2])
        df['r'] = df['training_params_string'].apply(lambda x: int(x.split('_')[3]))
        df['lora_alpha'] = df['training_params_string'].apply(lambda x: int(x.split('_')[4]))
        df['lora_dropout'] = df['training_params_string'].apply(lambda x: float(x.split('_')[5]))
        df['gradient_accumulation_steps'] = df['training_params_string'].apply(lambda x: int(x.split('_')[6]))
        df['learning_rate'] = df['training_params_string'].apply(lambda x: x.split('_')[7].replace('.csv', ''))
    elif df['fine_tuning'][0] in ['base', 'NoFT']:
        df['training_params_string'] = None
    return df

csv_files = glob.glob('data/evaluation_results/*.csv')
dfs = []
empty_n = 0
for file in csv_files:
    if file =='data/evaluation_results/joint_results.csv':
        continue
    tmp_df = pd.read_csv(file)
    if tmp_df.shape[0] == 0:
        print('EMPTY FILE: ', file)
        empty_n += 1
        continue
    print('SOURCE: ', file)
    tmp_df = extract_params_from_file_name(tmp_df)
    dfs.append(tmp_df)

res = pd.concat(dfs).to_csv('data/evaluation_results/joint_results.csv')
# res = res.drop(columns=['similar_is_equal', 'similar_is_equal_threshold'])
# print(empty_n)
# res.sort_values(by='f1_score', ascending=False)


SOURCE:  data/evaluation_results/mistral_noInstr_8bit_wordsLevelTrue_evaluation.csv
SOURCE:  data/evaluation_results/zefiro_4bit_FT.csv


IndexError: list index out of range

In [ ]:
tmp_df = pd.read_csv(file)
extract_params_from_file_name